# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [7]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [17]:
def check_neighbor(letter,clean_word,front):
    """This function checks if the neighbor of guessed letter is empty
        Letter: The letter to check 
        front: True: check the front neighbor; False: check the behind neighbor
        return: True or False
    """
    my_word = clean_word.replace('.','0')
    if front:
        check_str = '0'+letter
    else:
        check_str = letter + '0'
    return re.match(check_str,my_word)!=None

def Count(letter,front):
        """ Function return Counter dict
            letter: the checking letter
            front: True or False""" 
        if front:
            check_str = '.'+letter
        else:
            check_str = letter + '.'
        segments_filtered = [segment for segment in segments if re.match(check_str,segment)]
        c = collections.Counter(segments_filtered)
        return c
text_file = open("words_250000_train.txt","r")
full_dictionary = text_file.read().splitlines()
text_file.close()
#return combinations with two letters return a dict contains 2-letter segments

segments = []
for word in full_dictionary:
    for i in range(0,len(word)):
        segment = word[i:i+2]
        if len(segment) ==2:
            segments.append(segment)
print(segments[:100])
clean_word = 'e0e'
letter = 'e'
print(check_neighbor(letter,clean_word,front=True))
print(re.search('0e',clean_word))
print('0e' in segments)
print(Count('e',front=True))

['aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'as', 'aa', 'ac', 'ch', 'he', 'en', 'aa', 'ae', 'ee', 'aa', 'ag', 'aa', 'ah', 'he', 'ed', 'aa', 'ah', 'hs', 'aa', 'al', 'aa', 'al', 'le', 'es', 'su', 'un', 'nd', 'aa', 'al', 'li', 'ii', 'is', 'aa', 'al', 'ls', 'st', 'aa', 'am', 'aa', 'an', 'nd', 'da', 'ah', 'hl', 'aa', 'ao', 'aa', 'ap', 'ps', 'ss', 'aa', 'ar', 'aa', 'ar', 'ra', 'au', 'aa', 'ar', 'rd', 'dv', 'va', 'ar', 'rk', 'aa', 'ar', 'rd', 'dw', 'wo', 'ol', 'lf', 'aa', 'ar', 're', 'en', 'aa', 'ar', 'rg', 'gh', 'aa', 'ar', 'ri', 'ik', 'ka', 'aa', 'ar', 'ro', 'on', 'ni', 'ic', 'aa', 'ar']
False
<re.Match object; span=(1, 3), match='0e'>
False
Counter({'te': 26219, 're': 24800, 'le': 22135, 'ne': 19584, 'de': 13954, 'se': 13813, 'he': 11707, 'me': 11077, 've': 10593, 'pe': 10209, 'ce': 9023, 'ie': 8301, 'ge': 7326, 'be': 5974, 'ee': 5426, 'ke': 5188, 'ze': 3531, 'ae': 3350, 'fe': 3328, 'we': 3228, 'ue': 2259, 'oe': 1715, 'ye': 1380, 'je': 735, 'xe': 638, 'qe': 5})


In [22]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []

        self.segments = self.letter_combination_dict()
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        if not self.guessed_letters:
            # iterate through all of the words in the old plausible dictionary
            for dict_word in current_dictionary:
                # continue if the word is not of the appropriate length
                if len(dict_word) != len_word:
                    continue
                    
                # if dictionary word is a possible match then add it to the current dictionary
                if re.match(clean_word,dict_word):
                    new_dictionary.append(dict_word)
            
            # overwrite old possible words dictionary with updated version
            self.current_dictionary = new_dictionary
            
            
            # count occurrence of all characters in possible word matches
            full_dict_string = "".join(new_dictionary)
            
            c = collections.Counter(full_dict_string)
            sorted_letter_count = c.most_common()                   
            
            guess_letter = '!'
            
            # return most frequently occurring letter in all possible words that hasn't been guessed yet
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break
                
            # if no word matches in training dictionary, default back to ordering of full dictionary
            if guess_letter == '!':
                sorted_letter_count = self.full_dictionary_common_letter_sorted
                for letter,instance_count in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break            
            
            return guess_letter
        else:
            for letter in self.guessed_letters:
                print('checking letter',letter)
                # check neighbor
                for front in [True,False]:
                    if self.check_neighbor(letter,clean_word,front):
                        print('checking front',front)
                        print('checking neighbor')
                        # count the segment combination
                        c = self.Count(letter,front)
                        sorted_letter_count = c.most_common()
                        print('counting',sorted_letter_count)
                        # return the guess letter from the most prevalent segment in all possible words that hasn't been guessed yet
                        guess_letter = self.prevalent_letter(sorted_letter_count,front)
                        print(letter,guess_letter,front)
                        break
            return guess_letter

                    




    

    
    def check_neighbor(self,letter,clean_word,front):
        """This function checks if the neighbor of guessed letter is empty
           Letter: The letter to check 
           front: True: check the front neighbor; False: check the behind neighbor
           return: True or False
        """
        my_word = clean_word.replace('.','0')
        if front:
            check_str = '0'+letter
        else:
            check_str = letter + '0'
        return re.match(check_str,my_word)!=None
    def Count(self,letter,front):
        """ Function return Counter dict
            letter: the checking letter
            front: True or False""" 
        if front:
            check_str = '0'+letter
        else:
            check_str = letter + '0'
        segments_filtered = [segment for segment in self.segments if check_str == segment]
        c = collections.Counter(segments_filtered)
        return c
    
    def prevalent_letter(self,sorted_segment_count,front):
        """return the letter from the most frequently occurring segment in all possible words that hasn't been guessed yet"""
        for segment,_ in sorted_segment_count:
            if front:
                letter = segment[0]
            else:
                letter = segment[-1]
            if letter not in self.guessed_letters:
                return letter 
        return None 
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [23]:
api = HangmanAPI(access_token="09ee027a7d93f260d1e857fd7e6789", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [26]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 5c7cc1a4abd0. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '5c7cc1a4abd0', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ _ _ _ _ _ '}
checking letter e
checking front True
checking front False


UnboundLocalError: local variable 'guess_letter' referenced before assignment

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [5]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

HangmanAPIError: {'error': 'Failed to authenticate with token INSERT_YOUR_TOKEN_HERE'}